# Object and image properties detection using Amazon Rekognition

***
This notebook provides a walkthrough of [object detection API](https://docs.aws.amazon.com/rekognition/latest/dg/labels.html) in Amazon Rekognition to identify objects.
***

# Initialize stuff

In [ ]:
# Initialise Notebook
import boto3
from IPython.display import HTML, display, Image as IImage
from PIL import Image, ImageDraw, ImageFont
import time
import os

import sagemaker

# variables
data_bucket = sagemaker.Session().default_bucket()
role = sagemaker.get_execution_role()
awsRegion = boto3.session.Session().region_name

s3 = boto3.client('s3')
rekognition = boto3.client('rekognition')


print(f"SageMaker role is: {role}\nDefault SageMaker Bucket: s3://{data_bucket}")

# Detect objects in image
***

![object-detection-sample](../datasets/cars.jpg)

In [ ]:
image_s3_key = 'content-moderation-im/image-moderation/cars.jpg'
s3.upload_file('../datasets/cars.jpg', data_bucket, image_s3_key)

#### Call Rekognition to detect objects in the image

In [ ]:
# Call Amazon Rekognition to detect objects in the image
# https://docs.aws.amazon.com/rekognition/latest/dg/API_DetectLabels.html

detectLabelsResponse = rekognition.detect_labels(
    Image={
        'S3Object': {
            'Bucket': data_bucket,
            'Name': image_s3_key,
        }
    }
)

#### Review the raw JSON reponse from Rekognition

In [ ]:
# Show JSON response returned by Rekognition Labels API (Object Detection)
# In the JSON response below, you will see Label, detected instances, confidence score and additional information.

display(detectLabelsResponse)

#### Display list of detected unsafe objects}

In [ ]:
flaggedObjects = ["Car"]

for label in detectLabelsResponse["Labels"]:
    if(label["Name"] in flaggedObjects):
        print("Detected unsafe object:")
        print("- {} (Confidence: {})".format(label["Name"], label["Confidence"]))
        print("  - Parents: {}".format(label["Parents"]))

#### The code snippets below demonstrate how to use an open-source computer vision library, Pillow, to draw bounding box overlays on images. This allows you to visualize the location of an object in an image by highlighting the area with a box.

In [ ]:
# Define a function that will display image with bounded boxes around recognized objects
# We will call this function in next step
  
def drawBoundingBoxes (sourceImage, boxes):
    # blue, green, red, grey
    colors = ((255,255,255),(255,255,255),(76,182,252),(52,194,123))
    
    # Download image locally
    #imageLocation = tempFolder+os.path.basename(sourceImage)
    imageLocation = sourceImage
    #s3.download_file(bucketName, sourceImage, imageLocation)

    # Draws BB on Image
    bbImage = Image.open(imageLocation)
    draw = ImageDraw.Draw(bbImage)
    width, height = bbImage.size
    col = 0
    maxcol = len(colors)
    line= 3
    for box in boxes:
        x1 = int(box[1]['Left'] * width)
        y1 = int(box[1]['Top'] * height)
        x2 = int(box[1]['Left'] * width + box[1]['Width'] * width)
        y2 = int(box[1]['Top'] * height + box[1]['Height']  * height)
        
        draw.text((x1,y1),box[0],colors[col])
        for l in range(line):
            draw.rectangle((x1-l,y1-l,x2+l,y2+l),outline=colors[col])
        col = (col+1)%maxcol
    
    imageFormat = "PNG"
    ext = sourceImage.lower()
    if(ext.endswith('jpg') or ext.endswith('jpeg')):
        imageFormat = 'JPEG'

    bbImage.save(imageLocation,format=imageFormat)

    display(bbImage)

In [ ]:
# Show image and bounded boxes around detected objects
boxes = []
objects = detectLabelsResponse['Labels']
for obj in objects:
    for einstance in obj["Instances"]:
        boxes.append ((obj['Name'], einstance['BoundingBox']))
    
drawBoundingBoxes("../datasets/cars.jpg", boxes)


#### Amazon Rekognition Image returns information about image quality (sharpness, brightness, and contrast) for the entire image. Sharpness and brightness are also returned for the foreground and background of the image. Image Properties can also be used to detect dominant colors of the entire image, foreground, background, and objects with bounding boxes.

![image-property-detection-sample](../datasets/green_car.jpeg)

In [ ]:
image_property_s3_key = 'content-moderation-im/image-moderation/green_car.jpeg'
s3.upload_file('../datasets/cars.jpg', data_bucket, image_property_s3_key)

In [ ]:
imagePropertiesResponse = rekognition.detect_labels(
    Image={
        'S3Object': {
            'Bucket': data_bucket,
            'Name': image_property_s3_key,
        }
    },
    Features=["IMAGE_PROPERTIES"], # 'GENERAL_LABELS',
    Settings={
        "ImageProperties": {
            "MaxDominantColors": 10
        }
    }
)

In [ ]:
# Show JSON response returned by Rekognition Labels API (Object Detection)
# In the JSON response below, you will see Image Properties, sharpness, brightness, contrast and additional information.

display(imagePropertiesResponse)

In [ ]:
# Get image size and DPI (Dots per Inch) using Pillow
im = Image.open('../datasets/green_car.jpeg')
print(f'Image size: {im.size[0]} X {im.size[1]}')
print(f'DPI: {im.info["dpi"]}')

# Image quality from Rekognition
print('Quality: ')
print(f'     Brightness: {round(imagePropertiesResponse["ImageProperties"]["Quality"]["Brightness"],2)}%')
print(f'     Sharpness: {round(imagePropertiesResponse["ImageProperties"]["Quality"]["Sharpness"],2)}%')
print(f'     Contrast: {round(imagePropertiesResponse["ImageProperties"]["Quality"]["Contrast"],2)}%')

print('Dominant Colors: ')
for c in imagePropertiesResponse["ImageProperties"]["DominantColors"]:
    html = f'''<div style="background-color:{c['HexCode']};width:250px;margin-left:60px;padding-left:5px;">RGB({c['Red']},{c['Green']},{c['Blue']}])  {round(c["PixelPercent"],2)}%</div>'''
    display(HTML(html))

### Learn more about building an end to end worker safety app
To learn on how to build an end to end worker safety app, learn more at https://github.com/aws-samples/aws-deeplens-worker-safety-project

***
### References
- https://docs.aws.amazon.com/rekognition/latest/dg/API_DetectLabels.html
- https://docs.aws.amazon.com/rekognition/latest/dg/API_StartLabelDetection.html
- https://docs.aws.amazon.com/rekognition/latest/dg/API_GetLabelDetection.html

***

You have successfully used Amazon Rekognition to identify specific objects in images and videos. In the next module,nyou will learn how to recognize explicit, suggestive or violent content in the images and videos.